<a href="https://colab.research.google.com/github/joexu22/llama2-finetune/blob/main/Fine_tune_llama_2_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutoriel** - Fine-tuning Llama2

This code notebook makes it possible to fine-tune llama 2 to speak 17th century French, with an instruction dataset extracted from a few hunder French novels published from 1600 to 1700. This is a fun sample to check whether finetuning is working or not (and no, ChatGPT cannot do it).

Obviously you can use it for any dataset hosted on huggingface provided they have the ###human / ###assistant data structure.

# Installation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/llama"

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/My Drive/llama'
/content


We are going to use a script created by Younes Belkada to finetune Llama 2.

In [2]:
!rm -rf e90381ed142ee80c8e7ea602b18d50f0
!git clone https://gist.github.com/Pclanglais/e90381ed142ee80c8e7ea602b18d50f0

Cloning into 'e90381ed142ee80c8e7ea602b18d50f0'...
remote: Enumerating objects: 38, done.
remote: Total 38 (delta 0), reused 0 (delta 0), pack-reused 38
Receiving objects: 100% (38/38), 6.27 KiB | 6.27 MiB/s, done.
Resolving deltas: 100% (14/14), done.


We check for the dependences.

In [3]:
!pip install accelerate==0.21.0
!pip install peft==0.4.0
!pip install bitsandbytes==0.40.2
!pip install transformers==4.30.2
!pip install trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.2
    Uninstalling transformers-4.33.2:
      Successfully uninstalled transformers-4.33.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

We get inside the folder that contains the finetuning script.

In [4]:
%cd "e90381ed142ee80c8e7ea602b18d50f0"

/content/e90381ed142ee80c8e7ea602b18d50f0


We launch the script.

In [5]:
!python finetune_llama_v2.py --dataset_name "AlexanderDoria/novel17_test" --max_steps 500 --merge_and_push True --model_name "daryl149/llama-2-7b-chat-hf"

2023-09-23 23:07:38.041079: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100% 2/2 [00:19<00:00,  9.57s/it]
Extracting data files: 100% 2/2 [00:00<00:00, 2084.64it/s]
Generating train split: 1 examples [00:00, 29.65 examples/s]
Generating test split: 1 examples [00:00, 880.05 examples/s]
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Map: 100% 1/1 [00:00<00:00, 18.99 examples/s]
  0% 0/500 [00:00<?, ?it/s]You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
{'loss': 1.0826, 'learning_rate': 0.0002, 'epoch': 10.0}
{'loss': 1.0046, 'learning_rate': 0.0002, 'epoch

We are going to test the inference but I highly recommend to renew the runtime: otherwise your ram is very likely to be saturated.

Then we load the model

In [8]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("results/final_merged_checkpoint")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Then we generate some text (in old French). It helps if you give an headstart to the model (since fine-tuning is really rough and simple)

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("daryl149/llama-2-7b-chat-hf")

from transformers import pipeline

generator = pipeline(task="text-generation", model=model, tokenizer = tokenizer, max_length=200)

result = generator("Ecrivez un texte en ancien français du 17e siècle sur le meilleur moyen de voyager sur la lune ### Assistant: Je serois parti sur la Lune")

print(result)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'generated_text': "Ecrivez un texte en ancien français du 17e siècle sur le meilleur moyen de voyager sur la lune ### Assistant: Je serois parti sur la Lune.###  Buisson. Qui plus beau moyen fut jamais findré de voyager sur la lune que de faire une barque de cire d'un jour au lendemain et de la faire voler dans les airs jusque-lui donner la lune au visage. Car si elle est belle de son aspect terrestre, elle est encore plus belle de près, et les étoiles ne sont point si éloignées qu'on le pense. Si le vent ne vous empêche pas de prendre la barque, si la lune ne vous étonne point de sa clarté, si les étoiles ne vous éblouissent point de leur splendeur, vous verrez des choses mer"}]
